# 客户流失预测 - FLAML实现

> 🎯 目标：使用FLAML实现最快速的客户流失预测模型（1-3分钟）

**FLAML优势**:
- ⚡ 速度最快（成本优化算法）
- 💰 计算成本最低
- 📈 大数据集友好

---

In [ ]:
# 环境准备
import pandas as pd
import numpy as np
from flaml import AutoML
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
import time

# 加载数据
data_path = "../../../supervised_learning/projects/02_customer_churn_prediction/data/WA_Fn-UseC_-Telco-Customer-Churn.csv"
data = pd.read_csv(data_path)

# 预处理
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'], errors='coerce')
data = data.drop('customerID', axis=1).dropna()

# 分离特征和目标
X = pd.get_dummies(data.drop('Churn', axis=1), drop_first=True)
y = (data['Churn'] == 'Yes').astype(int)

# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"训练集: {X_train.shape}")
print(f"测试集: {X_test.shape}")

In [ ]:
# FLAML训练
automl = AutoML()

start_time = time.time()

automl.fit(
    X_train, y_train,
    task='classification',
    time_budget=120,         # 2分钟时间预算
    metric='roc_auc',        # 优化AUC
    estimator_list=['xgboost', 'lgbm', 'rf', 'extra_tree'],  # 只用树模型
    log_file_name='flaml_churn.log',
    seed=42
)

training_time = time.time() - start_time

print(f"\n✅ 训练完成！耗时: {training_time:.2f}秒")
print(f"最佳模型: {automl.best_estimator}")
print(f"最佳AUC: {1 - automl.best_loss:.4f}")
print(f"\n最佳配置: {automl.best_config}")

In [ ]:
# 测试集评估
y_pred = automl.predict(X_test)
y_pred_proba = automl.predict_proba(X_test)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)

print("测试集性能:")
print(f"Accuracy: {accuracy:.4f}")
print(f"AUC: {auc:.4f}")
print(f"\n分类报告:")
print(classification_report(y_test, y_pred))

In [ ]:
# 特征重要性
if hasattr(automl.model.estimator, 'feature_importances_'):
    import matplotlib.pyplot as plt
    
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': automl.model.estimator.feature_importances_
    }).sort_values('importance', ascending=False).head(15)
    
    plt.figure(figsize=(10, 6))
    plt.barh(feature_importance['feature'], feature_importance['importance'])
    plt.xlabel('Importance')
    plt.title('Top 15 Feature Importance (FLAML)')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

## 总结

**FLAML关键优势**:
1. ⚡ 速度极快（通常1-2分钟）
2. 📊 性能接近PyCaret
3. 💡 成本优化算法，资源高效

**下一步**: 查看 `03_comparison_with_manual.ipynb` 了解详细对比